In [ ]:
# %%capture
# # !pip install "unsloth==0.2.13" --force-reinstall
# !pip install datasets
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
# Add this as a new cell at the very beginning (Cell 0):
import torch
import gc

# Clear GPU cache
gc.collect()
torch.cuda.empty_cache()

# Reset CUDA context
torch.cuda.synchronize()
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"Memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

In [ ]:
%%capture
import os, re, gc
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth unsloth_zoo
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2
!pip install tensorboard tensorboardX

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    # load_in_4bit = load_in_4bit, #change this to load_in_8bit = True if 8bit config req
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state=42,
    loftq_config=None,
)

In [ ]:
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import gc
import random

In [ ]:
# Example dataset class for SafeLoRA
class SafetyDataset(Dataset):
    """
    Dataset class that includes safety labels for SafeLoRA.

    Args:
        texts: List of input texts
        labels: List of output labels (for language modeling, usually same as texts)
        safety_labels: List of safety labels (0 for safe, 1 for unsafe)
        tokenizer: Tokenizer for encoding texts
        max_length: Maximum sequence length
    """
    def __init__(self, texts, labels, safety_labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.safety_labels = safety_labels  # 0 for safe, 1 for unsafe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        safety_label = self.safety_labels[idx]

        # Tokenize
        encodings = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        # For causal language modeling, shift labels
        input_ids = encodings.input_ids.squeeze()
        attention_mask = encodings.attention_mask.squeeze()
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore pad tokens

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "safety_labels": safety_label,
        }

In [ ]:
# Function to load data from separate safe and unsafe CSV files
def load_data_from_csvs(safe_csv_path, unsafe_csv_path):
    """
    Load instruction-response pairs from separate safe and unsafe CSV files.

    Args:
        safe_csv_path: Path to the CSV file with safe instruction-response pairs
        unsafe_csv_path: Path to the CSV file with unsafe instruction-response pairs

    Returns:
        texts: List of texts for training (instruction + response)
        labels: List of target labels
        safety_labels: List of safety labels (0 for safe, 1 for unsafe)
    """
    import pandas as pd

    logger.info(f"Loading safe data from {safe_csv_path}")
    logger.info(f"Loading unsafe data from {unsafe_csv_path}")

    texts = []
    labels = []
    # sample_count = 0
    safety_labels = []

    try:
        # Read safe CSV file
        safe_df = pd.read_csv(safe_csv_path)

        # Check if required columns exist
        if 'instruction' not in safe_df.columns or 'response' not in safe_df.columns:
            raise ValueError("CSV must contain 'instruction' and 'response' columns")

        # Format the safe training data
        for _, row in safe_df.iterrows():
            instruction = str(row['instruction']).strip()
            response = str(row['response']).strip()

            # Format as a single text for training
            formatted_text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
            texts.append(formatted_text)
            safety_labels.append(0)  # Safe label

        # Read unsafe CSV file
        unsafe_df = pd.read_csv(unsafe_csv_path)

        # Check if required columns exist
        if 'instruction' not in unsafe_df.columns or 'response' not in unsafe_df.columns:
            raise ValueError("CSV must contain 'instruction' and 'response' columns")

        # Format the unsafe training data
        for _, row in unsafe_df.iterrows():
            instruction = str(row['instruction']).strip()
            response = str(row['response']).strip()

            # Format as a single text for training
            formatted_text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
            texts.append(formatted_text)
            safety_labels.append(1)  # Unsafe label

        # For causal LM, labels are the same as inputs
        labels = texts.copy()

        logger.info(f"Loaded {len(safe_df)} safe examples and {len(unsafe_df)} unsafe examples")
        logger.info(f"Total dataset size: {len(texts)} examples")

        return texts, labels, safety_labels

    except Exception as e:
        logger.error(f"Error loading CSV data: {e}")
        raise

In [ ]:

from datasets import load_dataset
def load_data_from_huggingface_datasets(safe_dataset, unsafe_dataset):
    """
    Alternate examples from safe and unsafe datasets and return texts, labels, and safety labels.
    """
    texts, labels, safety_labels = [], [], []

    min_len = min(len(safe_dataset), len(unsafe_dataset))
    for i in range(min_len):
        # Safe example
        s_inst = safe_dataset[i]['instruction'].strip()
        s_resp = safe_dataset[i]['response'].strip()
        s_text = f"### Instruction:\n{s_inst}\n\n### Response:\n{s_resp}"
        texts.append(s_text)
        labels.append(s_text)
        safety_labels.append(0)

        # Unsafe example
        u_inst = unsafe_dataset[i]['instruction'].strip()
        u_resp = unsafe_dataset[i]['response'].strip()
        u_text = f"### Instruction:\n{u_inst}\n\n### Response:\n{u_resp}"
        texts.append(u_text)
        labels.append(u_text)
        safety_labels.append(1)

    return texts, labels, safety_labels

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

def load_and_split_safe_unsafe_datasets(safe_dataset_name: str, unsafe_dataset_name: str, test_size: float = 0.1):
    safe_full = load_dataset(safe_dataset_name, split="train")
    unsafe_full = load_dataset(unsafe_dataset_name, split="train")

    safe_train_indices, safe_test_indices = train_test_split(
        list(range(len(safe_full))), test_size=test_size, random_state=42
    )
    unsafe_train_indices, unsafe_test_indices = train_test_split(
        list(range(len(unsafe_full))), test_size=test_size, random_state=42
    )

    def select(dataset, indices):
        return dataset.select(indices)

    return (
        select(safe_full, safe_train_indices),
        select(unsafe_full, unsafe_train_indices),
        select(safe_full, safe_test_indices),
        select(unsafe_full, unsafe_test_indices),
    )

In [ ]:
from transformers import default_data_collator

def custom_collator(features):
    batch = default_data_collator(features)
    if "safety_labels" in features[0]:
        batch["safety_labels"] = torch.tensor([f["safety_labels"] for f in features])
    return batch


In [ ]:
import os
from datetime import datetime

# TensorBoard setup - matching QLoRA_Baseline format
tensorboard_base_dir = "/home/ubuntu/DiskUsEast1/finetuning_evaluation/tensorboard_logs"
os.makedirs(tensorboard_base_dir, exist_ok=True)

# Create run-specific directory with timestamp
run_name = "SafeLoRA_Training"  # Descriptive name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
tensorboard_run_dir = os.path.join(tensorboard_base_dir, f"{run_name}_{timestamp}")

print(f"📊 TensorBoard logs will be saved to: {tensorboard_run_dir}")

In [ ]:
import os
import torch
import numpy as np
from sklearn.metrics import davies_bouldin_score, silhouette_score
from transformers import Trainer
from torch.utils.data import DataLoader
import gc


class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.step_count = 0

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        self.step_count += 1
        outputs = model(**inputs)
        loss = outputs.loss

        # Compute cluster metrics every N steps (without visualization)
        if self.step_count % 50 == 0:
            print(f"\n>>> Step {self.step_count}: Computing cluster metrics...")
            metrics = self.compute_cluster_metrics(model)

            # Log to TensorBoard
            if self.args.report_to == "tensorboard":
                self.log(metrics)

            print(f"[Step {self.step_count}] Loss: {loss.item():.4f}")
            if "DBS" in metrics:
                print(f"  DBS: {metrics['DBS']:.4f} (lower is better)")
                print(f"  Silhouette: {metrics['Silhouette']:.4f} (higher is better)")

        return (loss, outputs) if return_outputs else loss

    def get_eval_dataloader(self, eval_dataset=None):
        """Lightweight dataloader for metric computation"""
        if eval_dataset is None:
            eval_dataset = self.eval_dataset
        if eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")

        return DataLoader(
            eval_dataset,
            batch_size=2,  # Tiny batch size
            shuffle=False,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=0,  # No multiprocessing to save memory
            pin_memory=False,
        )

    def compute_cluster_metrics(self, model):
        """Compute DBS and Silhouette WITHOUT t-SNE visualization"""
        torch.cuda.empty_cache()
        gc.collect()

        model.eval()
        embeddings, labels = [], []
        sample_count = 0
        MAX_SAMPLES = 50  # Small subset

        dataloader = self.get_eval_dataloader()

        with torch.no_grad():
            for batch in dataloader:
                if sample_count >= MAX_SAMPLES:
                    break

                input_ids = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=True
                )

                # Extract last token embedding
                last_hidden_state = outputs.hidden_states[-1]
                embedding = last_hidden_state[:, -1, :].detach().cpu().numpy()

                safety_labels = batch["safety_labels"].cpu().numpy()
                embeddings.append(embedding)
                labels.extend(safety_labels)
                sample_count += len(safety_labels)

                # Clear memory immediately
                del input_ids, attention_mask, outputs, last_hidden_state

        torch.cuda.empty_cache()

        X = np.vstack(embeddings)
        labels = np.array(labels)

        metrics = {}
        if len(np.unique(labels)) >= 2:
            metrics["DBS"] = davies_bouldin_score(X, labels)
            metrics["Silhouette"] = silhouette_score(X, labels)

        # Cleanup
        del X, embeddings, labels
        torch.cuda.empty_cache()
        gc.collect()

        model.train()
        return metrics

In [ ]:
safe_train, unsafe_train, safe_test, unsafe_test = load_and_split_safe_unsafe_datasets(
    safe_dataset_name="utk6/safe_dataset_gretel",
    unsafe_dataset_name="utk6/unsafe_dataset_gretel",
    test_size=0.1
)

# Build alternated train and test datasets
train_texts, train_labels, train_safety_labels = load_data_from_huggingface_datasets(safe_train, unsafe_train)
test_texts, test_labels, test_safety_labels = load_data_from_huggingface_datasets(safe_test, unsafe_test)

# Tokenized Datasets
train_dataset = SafetyDataset(train_texts, train_labels, train_safety_labels, tokenizer, max_length=max_seq_length)
test_dataset = SafetyDataset(test_texts, test_labels, test_safety_labels, tokenizer, max_length=max_seq_length)
print("train size: ",len(train_dataset))
print("test size",len(test_dataset))
eval_dataset = None

In [ ]:
from datasets import Dataset

examples = []
for i in range(len(train_dataset)):
    item = train_dataset[i]
    examples.append({
        "input_ids": item["input_ids"],
        "attention_mask": item["attention_mask"],
        "labels": item["labels"],
        "safety_labels": item["safety_labels"]
    })

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_list(examples)

In [ ]:
from datasets import Dataset

examples = []
for i in range(len(test_dataset)):
    item = test_dataset[i]
    examples.append({
        "input_ids": item["input_ids"],
        "attention_mask": item["attention_mask"],
        "labels": item["labels"],
        "safety_labels": item["safety_labels"]
    })

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_list(examples)

In [ ]:
from trl import SFTTrainer, SFTConfig

# Training arguments with TensorBoard
training_args = SFTConfig(
    output_dir="./llama3_sft",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,  # Effective batch = 32
    warmup_steps=10,
    max_steps=250,  # 200-300 range
    learning_rate=2e-4,
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    save_steps=250,  # Save at end
    save_total_limit=1,
    bf16=True,
    fp16=False,

    # TensorBoard configuration
    report_to="tensorboard",
    logging_dir=tensorboard_run_dir,
    logging_first_step=True,
)

# Initialize trainer
trainer = CustomTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Keep for cluster metrics computation
    args=training_args,
    data_collator=custom_collator,
)

In [ ]:
# Show GPU stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# Train
trainer_stats = trainer.train()

In [ ]:
# Show stats after training
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")

In [ ]:
# Save LoRA adapters
save_dir = "lora_model"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"✅ Model saved to {save_dir}/")

# Also save to GGUF for Ollama (optional)
if True:
    model.save_pretrained_gguf("model", tokenizer, quantization_method="q8_0")
    print(f"✅ GGUF model saved to model/")